In [36]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

In [22]:
def front_pareto(population, evaluations):
    nup = population.shape[0]
    domination_count = np.zeros(nup)
    pareto_front = np.empty(nup)
    dominatees = []
    f = []
    for i in xrange(nup):
        dominatees.append(np.nonzero(np.all(evaluations > evaluations[i], axis=1))[0])
        domination_count[i] = np.sum(np.all(evaluations < evaluations[i], axis=1))
        if domination_count[i] == 0:
            pareto_front[i] = 1
            f.append(i)
    
    fr = 2
    while f:
        nxt = []
        for p in f:
            for d in dominatees[p]:
                domination_count[d] -= 1
                if domination_count[d] == 0:
                    pareto_front[d] = fr
                    nxt.append(d)
        fr += 1
        f = nxt
    return pareto_front

In [154]:
def NSGA2_sort(population, evaluations):
    nup = population.shape[0]
    nf = evaluations.shape[1]
    pareto_front = front_pareto(population, evaluations)
    distances = np.zeros(nup)
        
    for i in xrange(nf):
        idxs = np.argsort(evaluations[:, i])
        population = population[idxs]
        evaluations = evaluations[idxs]
        pareto_front = pareto_front[idxs]
        distances = distances[idxs]
        distances[0] = np.inf
        distances[nup-1] = np.inf
        distances[1:nup-1] += evaluations[2:, i] - evaluations[:-2, i]

    idxs = np.lexsort((-distances, pareto_front))
    return population[idxs], evaluations[idxs]

In [113]:
def tournament_selection(population_size, how_many, t_size = 5):
    return np.random.choice(population_size, how_many * t_size).reshape((how_many, t_size)).min(axis=1)

In [114]:
def NSGA2(iterations, genome_length, population_size, offspring_size, evaluation_functions, ranges, mutation_chance, mutation_range):
    
    log_populations = np.empty((iterations, population_size, genome_length))
    
    probs = np.arange(population_size, dtype=float)[::-1]
    probs = probs/np.sum(probs)
    
    lower_bounds = ranges[:, 0]
    upper_bounds = ranges[:, 1]
    
    sigma = mutation_range
    
    current_population_solutions = np.random.rand(population_size, genome_length) * (upper_bounds - lower_bounds) + lower_bounds
    current_population_scores = np.empty((population_size, len(evaluation_functions)))
    
    for f in xrange(len(evaluation_functions)):
        current_population_scores[:, f] = evaluation_functions[f](current_population_solutions).T
        
    current_population_solutions, current_population_scores = NSGA2_sort(current_population_solutions, current_population_scores)
    
    for i in tqdm_notebook(xrange(iterations)):
        # parent selection
        par1Idx = tournament_selection(population_size, offspring_size)
        par2Idx = tournament_selection(population_size, offspring_size)
        #par1Idx = np.random.choice(population_size, offspring_size, p=probs)
        #par2Idx = np.random.choice(population_size, offspring_size, p=probs)
        
        # crossover
        alphas = np.random.rand(offspring_size, genome_length)
        offspring_population_solutions = alphas * current_population_solutions[par1Idx] + (1 - alphas) * current_population_solutions[par2Idx]
        
        # mutation
        mutation_indicator = np.random.rand(offspring_size, genome_length) < mutation_chance
        offspring_population_solutions += mutation_indicator * sigma * (np.random.rand(offspring_size, genome_length)*2 - 1)
        np.clip(offspring_population_solutions, lower_bounds, upper_bounds, out=offspring_population_solutions)
        
        # evaluation of offspring
        offspring_population_scores = np.empty((offspring_size, len(evaluation_functions)))
        for f in xrange(len(evaluation_functions)):
            offspring_population_scores[:, f] = evaluation_functions[f](offspring_population_solutions).T
        
        combined_solutions, combined_evaluations = NSGA2_sort(np.vstack([current_population_solutions, offspring_population_solutions]), np.vstack([current_population_scores, offspring_population_scores]))
        current_population_solutions = combined_solutions[:population_size]
        current_population_scores = combined_evaluations[:population_size]
        log_populations[i, :, :] = current_population_solutions[:, :]
        
    return log_populations
        

In [145]:
def graph_population(population, evaluation_functions):
    scores = np.empty((population.shape[0], len(evaluation_functions)))
    for f in xrange(len(evaluation_functions)):
        scores[:, f] = evaluation_functions[f](population).T
    
    par = front_pareto(population, scores)
    lowest = scores[par == 1]
    
    plt.plot(figsize=(18, 6))
    plt.scatter(scores[:, 0], scores[:, 1], c=par)
    plt.plot(lowest[:, 0], lowest[:, 1], 'ro')
    plt.xlabel('1st fitness function')
    plt.ylabel('2nd fitness function')
    plt.show()

## SCH problem

In [133]:
def SCH1(pop):
    return pop * pop

def SCH2(pop):
    p = pop - 2
    return p*p

In [157]:
iteration_count = 100
genome_length = 1
population_size = 300
offspring_count = 900
ev_fun_sch = np.array([SCH1, SCH2])
ranges = np.array([[-1000, 1000]])
mutation_chance = 0.1
mutation_strength = 0.1
pops_sch = NSGA2(iteration_count, genome_length, population_size, offspring_count, ev_fun_sch, ranges, mutation_chance, mutation_strength)

HBox(children=(IntProgress(value=0), HTML(value=u'')))

In [158]:
def f_sch(x):
    graph_population(pops_sch[x], ev_fun_sch)
    
x = interact(f_sch, x=widgets.IntSlider(min=0,max=99,step=1,value=0))


interactive(children=(IntSlider(value=0, description=u'x', max=99), Output()), _dom_classes=('widget-interact',))

## FON

In [130]:
def FON1(pop):
    return 1 - np.exp(1 - np.sum((pop - 1/np.sqrt(3))**2, axis=1))

def FON2(pop):
    return 1 - np.exp(1 - np.sum((pop + 1/np.sqrt(3))**2, axis=1))

In [159]:
iteration_count = 100
genome_length = 3
population_size = 300
offspring_count = 900
ev_fun_fon = np.array([FON1, FON2])
ranges = np.array([[-4, 4]])
mutation_chance = 0.1
mutation_strength = 0.01
pops_fon = NSGA2(iteration_count, genome_length, population_size, offspring_count, ev_fun_fon, ranges, mutation_chance, mutation_strength)

HBox(children=(IntProgress(value=0), HTML(value=u'')))

In [160]:
def f_fon(x):
    graph_population(pops_fon[x], ev_fun_fon)
    
x = interact(f_fon, x=widgets.IntSlider(min=0,max=99,step=1,value=0))

interactive(children=(IntSlider(value=0, description=u'x', max=99), Output()), _dom_classes=('widget-interact',))

## ZTD1

In [123]:
def ZTD11(pop):
    return pop[:, 0]

def ZTD1g(pop):
    return 1 + 9 * np.mean(pop[:, 1:], axis=1)

def ZTD12(pop):
    g = ZTD1g(pop)
    return g * (1 - np.sqrt(pop[:, 0]/g))

In [161]:
iteration_count = 100
genome_length = 30
population_size = 300
offspring_count = 900
ev_fun_ztd1 = np.array([ZTD11, ZTD12])
ranges = np.array([[0, 1]])
mutation_chance = 0.1
mutation_strength = 0.05
pops_ztd1 = NSGA2(iteration_count, genome_length, population_size, offspring_count, ev_fun_ztd1, ranges, mutation_chance, mutation_strength)

HBox(children=(IntProgress(value=0), HTML(value=u'')))

In [162]:
def f_ztd1(x):
    graph_population(pops_ztd1[x], ev_fun_ztd1)
    
x = interact(f_ztd1, x=widgets.IntSlider(min=0,max=99,step=1,value=0))

interactive(children=(IntSlider(value=0, description=u'x', max=99), Output()), _dom_classes=('widget-interact',))

## ZTD3

In [148]:
def ZTD31(pop):
    return pop[:, 0]

def ZTD3g(pop):
    return 1 + 9 * np.mean(pop[:, 1:], axis=1)

def ZTD32(pop):
    g = ZTD3g(pop)
    return g * (1 - np.sqrt(pop[:, 0]/g) - ((pop[:, 0] * np.sin(10*np.pi*pop[:, 0]))/g))

In [166]:
iteration_count = 100
genome_length = 30
population_size = 600
offspring_count = 2000
ev_fun_ztd3 = np.array([ZTD31, ZTD32])
ranges = np.array([[0, 1]])
mutation_chance = 0.1
mutation_strength = 0.1
pops_ztd3 = NSGA2(iteration_count, genome_length, population_size, offspring_count, ev_fun_ztd3, ranges, mutation_chance, mutation_strength)

HBox(children=(IntProgress(value=0), HTML(value=u'')))

In [167]:
def f_ztd3(x):
    graph_population(pops_ztd3[x], ev_fun_ztd3)
    
x = interact(f_ztd3, x=widgets.IntSlider(min=0,max=99,step=1,value=0))

interactive(children=(IntSlider(value=0, description=u'x', max=99), Output()), _dom_classes=('widget-interact',))